In [1]:
import numpy as np
import gym
from collections import deque
import random
import sys
import pandas as pd
import matplotlib.pyplot as plt
from numpy import cos, sin
from gym import spaces
from gym.error import DependencyNotInstalled
from typing import Optional
from control.matlab import ss, lsim, linspace, c2d
from functools import partial
import math
import gym
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import random
from sklearn.preprocessing import StandardScaler
import math

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def angle_normalize(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi

In [4]:
class Pend1(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 30,
    }

    def __init__(self, render_mode: Optional[str] = None, g=10.0):
        super(Pend1, self).__init__()
        self.max_speed = 8
        self.max_torque = 2.0
        self.dt = 0.05
        self.g = g
        self.m = 1.0
        self.l = 1.0
        self.center = np.array([1,0,0])
        self.obstacle = np.array([0,1,3])
        self.render_mode = render_mode
        self.screen_dim = 500
        self.screen = None
        self.clock = None
        self.isopen = True

        high = np.array([1.0, 1.0, self.max_speed], dtype=np.float32)
        # This will throw a warning in tests/envs/test_envs in utils/env_checker.py as the space is not symmetric
        #   or normalised as max_torque == 2 by default. Ignoring the issue here as the default settings are too old
        #   to update to follow the openai gym api
        self.action_space = spaces.Box(
            low=-self.max_torque, high=self.max_torque, shape=(1,), dtype=np.float32
        )
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        # store current trace
        self.cache1 = []
        self.cache2 = []
        self.cache3 = []
        # ball radius
        self.target_norm_radius = 0.2 # norm ball radius of target, tune this
        self.safe_norm_radius = 0.1 # norm ball radius of safe, tune this
        self.total_time = 120
        # step number
        self.steps = 0
        self.u_lowbound = None
        self.caches = []
        self.reward_cache = [] # cache distances to target norm ball center
#         self.avoid_reward_cache = [] # cache distances to obstacles norm ball center
        self.final_reward_cache = [] # cache final reward
        # How long should this trace be, i.e. deadline
        self.step_const = random.randint(10, 50)
        # Maximum reward from each trace
        self.max_reward_list = []
        self.quality_list = []
        self.total_steps = 0
        self.step_history = []
        self.reached = False
    def angle_normalize(x):
        return ((x + np.pi) % (2 * np.pi)) - np.pi
    def step(self, u):
        th, thdot = self.state  # th := theta
        g = self.g
        m = self.m
        l = self.l
        dt = self.dt
        # simulate next step and get measurement
        self.steps += 1
        self.total_steps += 1
        terminated = False
        u = np.clip(u, -self.max_torque, self.max_torque)[0]
        self.last_u = u
        costs = 1
        self.last_u = u  # for rendering
        costs = angle_normalize(th) ** 2 + 0.1 * thdot**2 + 0.001 * (u**2)
                                        
        newthdot = thdot + (3 * g / (2 * l) * np.sin(th) + 3.0 / (m * l**2) * u) * dt
        newthdot = np.clip(newthdot, -self.max_speed, self.max_speed)
        newth = th + newthdot * dt

        self.state = np.array([newth, newthdot])

        # calculate euclidean distance and update reward cache
        dist = np.linalg.norm(self._get_obs() - self.center)
        obs_dist = np.linalg.norm(self._get_obs() - self.obstacle)
        reward = self.target_norm_radius - dist
#         obs_reward = obs_dist-self.safe_norm_radius

        self.reward_cache.append(reward)
#         self.avoid_reward_cache.append(obs_reward)
        # quantitative semantics
        # reach reward, encourage reaching target
        if self.steps < 10:
            reach_reward = max(self.reward_cache)
        else:
            reach_reward = max(self.reward_cache[-10:])
#         self.avoid_reward_cache.append(obs_reward)
        # quantitative semantics
        # reach reward, encourage reaching target
#         if self.steps < 10:
#             reach_reward = max(self.reward_cache)
#         else:
#             reach_reward = max(self.reward_cache[-10:])
#         if self.steps < 10:
#             avoid_reward = min(self.avoid_reward_cache)
#         else:
#             avoid_reward = min(self.avoid_reward_cache[-10:])

#         # very strict reward, always within target
#         strict_avoid_reward = avoid_reward - 0.5 * self.safe_norm_radius # half safe norm radius
#         strict_reach_reward = reach_reward - 0.5 * self.target_norm_radius # half target norm radius

        # overall reward, pick one of the final_reward
#         final_reward = reach_reward
#         final_reward = approach_reward
#         final_reward = min(reach_reward, avoid_reward) # reach and avoid
#         final_reward = min(approach_reward, avoid_reward) # approach and avoid
#         final_reward = min(reach_reward, approach_reward) # reach and approach
#         deadline_reward = (self.last_dist-dist)/(self.step_const - self.steps+1)
        final_reward = reach_reward
        # split cases: if already inside target, give very large constant reward for maintaining
        if dist <= self.target_norm_radius:
            final_reward = 10 # this gives 39/50 sucess with reach+approach+avoid

        self.final_reward_cache.append(final_reward)

        # If this is the last step, reset the state
        if self.steps == self.step_const or obs_dist<=self.safe_norm_radius or dist>5:
            self.max_reward_list.append(max(self.final_reward_cache)) # use max final reward to measure episodes
            self.step_history.append(self.total_steps)
            self.quality_list.append(sum(self.final_reward_cache))
            terminated = True
            self.reset()

#         # If within target norm ball, early terminate
#         if dist <= self.target_norm_radius:
#             terminated = True
#             self.reset()

        # Return next state, reward, done, info
        return self._get_obs(), final_reward, terminated, {}

    def reset(self):
        self.state = np.array([math.pi/4, -1])
        self.reward_cache = []
        self.final_reward_cache = []
        self.steps=0
        self.caches.append(self.cache1)
        self.caches.append(self.cache2)
        self.caches.append(self.cache3)
        self.cache1 = []
        self.cache2 = []
        self.cache3 = []
        # random # of steps for this trace
#         self.step_const = random.randint(10, 50) # deadline range
        self.step_const = 500
        self.reached = False
        return self._get_obs() # return something matching shape

    def _get_obs(self):
        theta, thetadot = self.state
        return np.array([np.cos(theta), np.sin(theta), thetadot], dtype=np.float32)

    def render(self):
        return

    def close(self):
        if self.screen is not None:
            import pygame
            pygame.display.quit()
            pygame.quit()
            self.isopen = False


In [21]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

#settings
target_norm_radius = 0.2 # norm ball radius of target, tune this
safe_norm_radius = 0.1
kernel = 1.0 * RBF(1.0)
num_epi= 200
num_steps = 200
action_list = [-1,1]
delta_tolerance = 0.5
discount = 0.9
center = np.array([1, 0, 0])
obstacles = np.array([0, 1, 3])
rmax = target_norm_radius
mean_offset = target_norm_radius/(1-discount)
# initial the env
env = Pend1()
state = env.reset()
lpschitz = 2
#Initial data collection
initial_buffer = 50
epsilon = 0.05
data_buffer = [[],[]]
for i in range(initial_buffer):
    this_action = random.randint(0,1)
    new_state, reward, done, _= env.step([action_list[this_action]])
    data_buffer[this_action].append((state, reward))
    state = new_state

In [22]:
trainx_1 = np.array([tuple[0] for tuple in data_buffer[0]])
trainx_2 = np.array([tuple[0] for tuple in data_buffer[1]])
trainy_1 = np.array([tuple[1] for tuple in data_buffer[0]])
trainy_2 = np.array([tuple[1] for tuple in data_buffer[1]])
train_x_list = [trainx_1, trainx_2]
train_y_list = [trainy_1, trainy_2]
x_scaler_list = []
for i in range(len(action_list)):
    x_scaler = StandardScaler()
    train_x_list[i] = x_scaler.fit_transform(train_x_list[i])
    x_scaler_list.append(x_scaler)

In [23]:
gpc1 = GaussianProcessRegressor(kernel=kernel, random_state=0)
gpc2 = GaussianProcessRegressor(kernel=kernel, random_state=0)
reward_q1 = []
reward_q2 = []
gpclist = [gpc1,gpc2]
reward_q_list = [reward_q1,reward_q2]
i = 0
for i in range(len(action_list)):
    gpclist[i].fit(train_x_list[i], train_y_list[i])
results = [0]*2
new_results = [0]*2
std_list_1 = [0]*2

D:\Anaconda\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [24]:
i = 0
for i in range(num_epi):
    print(i)
    for j in range(num_steps):
        # max q_st
        for k in range(len(reward_q_list)):
            state_entry = state.reshape(1,-1)
            state_entry_scale = x_scaler_list[k].transform(state_entry)
            if reward_q_list[k] == []:
                results[k], std_list_1[k] = gpclist[k].predict(state_entry_scale, return_std=True)
            else:
                lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - state_entry) for point in reward_q_list[k]]
                results[k], std_list_1[k] = gpclist[k].predict(state_entry_scale, return_std=True)
                results[k] = min(min(lp_values), results[k])
                
#         if reward_q1 == []:
#             state_entry = state.reshape(1,-1)
#             value_result1, std1 = gpc1.predict(state_entry, return_std=True)
#         else:
#             state_entry = state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - state_entry) for point in reward_q1]
#             value_result1, std1 = gpc1.predict(state_entry, return_std=True)
#             value_result1 = min(min(lp_values), value_result1)
#         if reward_q2 == []:
#             state_entry = state.reshape(1,-1)
#             value_result2, std2 = gpc2.predict(state_entry, return_std=True)
#         else:
#             state_entry = state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - state_entry) for point in reward_q2]
#             value_result2, std2 = gpc2.predict(state_entry, return_std=True)
#             value_result2 = min(min(lp_values), value_result2)        
#         if reward_q3 == []:
#             state_entry = state.reshape(1,-1)
#             value_result3, std3 = gpc3.predict(state_entry, return_std=True)
#         else:
#             state_entry = state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - state_entry) for point in reward_q3]
#             value_result3, std3 = gpc3.predict(state_entry, return_std=True)
#             value_result3 = min(min(lp_values), value_result3)        
#         if reward_q4 == []:
#             state_entry = state.reshape(1,-1)
#             value_result4, std4 = gpc4.predict(state_entry, return_std=True)
#         else:
#             state_entry = state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - state_entry) for point in reward_q4]
#             value_result4, std4 = gpc4.predict(state_entry, return_std=True)
#             value_result4 = min(min(lp_values), value_result4)
        # get actions
        # get new values
#         results = np.array([value_result1, value_result2, value_result3, value_result4])
#         results = np.array([x+mean_offset for x in results])
        which_action = action_list[np.argmax(results)]
        new_state, reward, done, _ = env.step([which_action])
        k = 0
        for k in range(len(reward_q_list)):
            new_state_entry = new_state.reshape(1,-1)
            new_state_entry_scale = x_scaler_list[k].transform(new_state_entry)
            if reward_q_list[k] == []:
                new_results[k], std_list_1[k] = gpclist[k].predict(new_state_entry_scale, return_std=True)
            else:
                lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - new_state_entry) for point in reward_q_list[k]]
                new_results[k], std_list_1[k] = gpclist[k].predict(new_state_entry_scale, return_std=True)
                new_results[k] = min(min(lp_values), new_results[k])
                
        
#         if reward_q1 == []:
#             new_state_entry = new_state.reshape(1,-1)
#             new_value_result1 = gpc1.predict(new_state_entry)[0]
#         else:
#             new_state_entry = new_state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - new_state_entry) for point in reward_q1]
#             new_value_result1 = gpc1.predict(new_state_entry)[0]
#             new_value_result1 = min(min(lp_values), new_value_result1)
#         if reward_q2 == []:
#             new_state_entry = new_state.reshape(1,-1)
#             new_value_result2 = gpc2.predict(new_state_entry)[0]
#         else:
#             new_state_entry = new_state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - new_state_entry) for point in reward_q2]
#             new_value_result2 = gpc2.predict(new_state_entry)[0]
#             new_value_result2 = min(min(lp_values), new_value_result2)        
#         if reward_q3 == []:
#             new_state_entry = new_state.reshape(1,-1)
#             new_value_result3 = gpc3.predict(new_state_entry)[0]
#         else:
#             new_state_entry = new_state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - new_state_entry) for point in reward_q3]
#             new_value_result3 = gpc3.predict(new_state_entry)[0]
#             new_value_result3 = min(min(lp_values), new_value_result3)        
#         if reward_q4 == []:
#             new_state_entry = state.reshape(1,-1)
#             new_value_result4 = gpc4.predict(new_state_entry)[0]
#         else:
#             new_state_entry = state.reshape(1,-1)
#             lp_values = [point[1]+lpschitz*np.linalg.norm(point[0] - new_state_entry) for point in reward_q4]
#             new_value_result4 = gpc4.predict(new_state_entry)[0]
#             new_value_result4 = min(min(lp_values), new_value_result4)
#         # get the q value
#         new_results = np.array([new_value_result1, new_value_result2, new_value_result3, new_value_result4])
#         new_results = np.array([x+mean_offset for x in new_results])
        
        q_value = reward+discount*max(new_results)
        action_index = np.argmax(results)
        var1 = std_list_1[action_index]**2
#         if which_action == -10:
#             var1 = std1*std1
#         elif which_action == -5:
#             var1 = std2*std2
#         elif which_action == 5:
#             var1 = std3*std3
#         else:
#             var1 = std4*std4
        if var1>delta_tolerance:
            if isinstance(q_value, np.ndarray):
                q_value = q_value[0]
            train_x_list[action_index] = np.vstack([train_x_list[action_index], state_entry])
            train_y_list[action_index] = np.concatenate([train_y_list[action_index], [q_value]])
            x_scaler = StandardScaler()
            train_x_list[action_index] = x_scaler.fit_transform(train_x_list[action_index])
            x_scaler_list[action_index] = x_scaler
            gpclist[action_index].fit(train_x_list[action_index], train_y_list[action_index])
        state_entry_new_scale = x_scaler_list[action_index].transform(state_entry)
        useless_result,update_std = gpclist[action_index].predict(state_entry_new_scale, return_std=True)
        var2 = update_std*update_std
        if (var1>delta_tolerance) and delta_tolerance > var2 and(results[action_index]-useless_result)>2*epsilon:
            reward_q_list[action_index].append((state, useless_result+epsilon))
            for k in range(len(action_list)):
                gpclist[k] = GaussianProcessRegressor(kernel=kernel, random_state=0)
                train_x_list[k] = train_x_list[k][:2]
                train_y_list[k] = train_y_list[k][:2]
        state= new_state
            

0


D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

1


D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

...

D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

34


D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimensio

35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [28]:
# Testing
# Test 50 traces
env = Pend1()
state = env.reset()
dims0 = []
dims1 = []
dims2 = []
euclids = []
center = np.array([1, 0, 0])
obstacle = np.array([0, 1, 3])
num_reached = 0
unsafe = 0
time = 0
stay = 0
for i in range(len(action_list)):
    reward_q_list[i].append((np.array([0,0,0]), np.array([-5])))
for j in range(10):
    dim0 = []
    dim1 = []
    dim2 = []
    euclid = []
    state = env.reset()
    flag = 0
    # Print initial state
    for i in range(50):
        q1_euclids = np.array([np.linalg.norm(state[:3]-x[0])for x in reward_q1])
        q2_euclids = np.array([np.linalg.norm(state[:3]-x[0])for x in reward_q2])
        close_1 = np.argmin(q1_euclids)
        close_2 = np.argmin(q2_euclids)
        close_value_list = np.array([reward_q1[close_1][1],reward_q2[close_2][1]])
        action = [action_list[np.argmin(close_value_list)]]
        new_state, reward, done, _ = env.step(action)
        dim0.append(state[0])
        dim1.append(state[1])
        dim2.append(state[2])
        dist = np.linalg.norm(state[:3]-center)
        obs_dist = np.linalg.norm(state[:3]-obstacle)
        euclid.append(dist)
        state = new_state
        if obs_dist <= 0.1:
            unsafe+=1
            time+=50
            break
        if dist <= 0.2: # stop
            if flag == 0:
                num_reached += 1
                time+=i
                flag = 1
            else:
                if i == 49:
                    stay+=1
                continue
    dims0.append(dim0)
    dims1.append(dim1)
    dims2.append(dim2)
    euclids.append(euclid)
ref= [math.pi/2]*100
time+=(1000-num_reached-unsafe)*50
print("Total number reached = " + str(num_reached))
print("Total number collison = " + str(unsafe))
print("Total number Stay = " + str(stay))
print("Average reach time = " + str(time/1000))

Total number reached = 0
Total number collison = 0
Total number Stay = 0
Average reach time = 50.0


In [27]:
len(reward_q2)

2997

In [ ]:
reward_q1

In [ ]:
plt.plot(ref)
for i in range(50):
    plt.plot(euclids[i])
plt.axhline(y=0.5, color='g', linestyle='--')
plt.axhline(y=0, color='g', linestyle='--')
plt.plot()
plt.xlabel('Time steps')
plt.ylabel('Euclidean distance to target')
plt.show()

In [41]:
import joblib
np.save('q_tab_pen_asap_500', reward_q_list)

D:\2022_conda\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


['dc_gpc2_ds.pkl']